In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from tensorflow.keras import regularizers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Embedding, Input
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,BatchNormalization
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
import tensorflow
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

In [ ]:
data_pretrained = pd.read_csv('../input/data-processed-6/data_processed.csv')

In [ ]:
num_of_classes = data_pretrained['Category_Last'].unique().shape[0]
num_of_classes

In [ ]:
X, y = data_pretrained['text_clean'],data_pretrained['Category_Last']
max_features = 7500
maxlen = 100
tokenizer = text.Tokenizer(num_words=max_features,filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~')


X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.08, random_state=66)
y_train,y_test = to_categorical(y_train,num_classes=num_of_classes),to_categorical(y_test,num_classes=num_of_classes)

X_train= X_train.astype("str")
X_test= X_test.astype("str")

tokenizer.fit_on_texts((X_train))#tokenizerın sadece train datasını görüp tokenleri de train datasına 
#göre çıkarmasını sağlıyor



#kelimeler tokenlere dönüşüyor
x_train_ = sequence.pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=maxlen,padding='post') 
x_test_ = sequence.pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=maxlen,padding='post')

In [ ]:
def base_model():
    #max_features = 10000
    #maxlen = 100
    #sgd = tensorflow.keras.optimizers.Adam()
    sgd = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
    embed_size = 32
    
    inp = Input(shape=(maxlen, ))
    
    #word_vectors = KeyedVectors.load_word2vec_format('../input/wordvec/trmodel', binary=True)        

    #weights = word_vectors.vectors

    
    
    x = Embedding(max_features, embed_size)(inp)
    
    #x = Embedding(max_features,word_vectors,trainble = False)(inp)
    
    
    
    x = Bidirectional(LSTM(25, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    #x = Dropout(0.3)(x)
    
    x = Dense(512,activation = 'relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(num_of_classes, activation="softmax",kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5))(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy',
                    optimizer=sgd,
                    metrics=['accuracy'])
    return model
model = base_model()

In [ ]:
model = base_model()
model.fit(x_train_,y_train,batch_size = 16,epochs = 10,validation_split=0.05)

In [ ]:
y_pred = model.predict(x_test_)
y_pred_ = np.argmax(y_pred, axis=1)

In [ ]:
target_names = ['Field Service-Diğer', 'Banking Applications & Software-Finsoft',
       'Field Service-Printer', 'Banking Applications & Software-BOSS',
       'Field Service-Software', 'Field Service-XNext',
       'Field Service-FINSOFT', 'Banking Applications & Software-xNext',
       'Field Service-PC', 'request-account locked',
       'Software-Operating System']
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test,axis = 1), y_pred_))
print('Classification Report')

print(classification_report(np.argmax(y_test,axis = 1), y_pred_, target_names=target_names))







In [ ]:
y_test